<a href="https://colab.research.google.com/github/CelikAbdullah/deep-learning-notebooks/blob/main/Computer%20Vision/models/SqueezeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras

# SqueezeNet

## The stem component

In [2]:
def stem(inputs):
  ''' Build the stem component of the SqueezeNet.
      inputs: the input tensor
  '''

  # a strided 7x7 conv layer followed by a 3x3 max pooling
  x = keras.layers.Conv2D(filters=96, kernel_size=7, strides=2, padding='same', activation='relu', kernel_initializer='glorot_uniform')(inputs)
  x = keras.layers.MaxPooling2D(3, strides=2)(x)

  return x

## The learner component

### Fire Block

In [3]:
def fire_block(x, n_filters):
  ''' Build a Fire Block.
	    x        : input to the block
      n_filters: number of filters
  '''
  # create the squeeze layer
  squeeze = keras.layers.Conv2D(filters=n_filters, kernel_size=1, strides=1, activation='relu', padding='same', kernel_initializer='glorot_uniform')(x)

  # branch the squeeze layer into a 1x1 and 3x3 convolution and double the number of filters
  expand1x1 = keras.layers.Conv2D(filters=n_filters * 4, kernel_size=1, strides=1, activation='relu', padding='same', kernel_initializer='glorot_uniform')(squeeze)
  expand3x3 = keras.layers.Conv2D(filters=n_filters * 4, kernel_size=3, strides=1, activation='relu', padding='same', kernel_initializer='glorot_uniform')(squeeze)

  # concatenate the feature maps from the 1x1 and 3x3 branches
  x = keras.layers.Concatenate()([expand1x1, expand3x3])

  return x

### Fire Group

In [4]:
def group(x, filters):
  ''' Construct a Fire Group
      x       : input to the group
      filters : list of number of filters per fire block (module)
  '''

  # add the fire blocks (modules) for this group
  for n_filters in filters:
    x = fire_block(x, n_filters)

  # a 3x3 max pooling to perform delayed downsampling
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)

  return x

### Learner component

In [5]:
def learner(x):
  ''' Build the learner component.
   	  x    : input to the learner
  '''
  # create the first fire group, progressively increase number of filters
  x = group(x, [16, 16, 32])
  # create the Second fire group
  x = group(x, [32, 48, 48, 64])
  # create the last fire block (module)
  x = fire_block(x, 64)
  # apply dropout; it is delayed to end of fire groups
  x = keras.layers.Dropout(0.5)(x)

  return x

## The task component

In [6]:
def task(x, n_classes):
  ''' Build the task component of SqueezeNet.
	    x        : input to the classifier
	    n_classes: number of output classes
  '''

  # set the number of filters equal to number of classes
  x = keras.layers.Conv2D(filters=n_classes, kernel_size=1, strides=1, activation='relu', padding='same', kernel_initializer='glorot_uniform')(x)
  # reduce each filter (class) to a single value
  x = keras.layers.GlobalAveragePooling2D()(x)
  # softmax activation layer
  x = keras.layers.Activation('softmax')(x)

  return x

## The SqueezeNet model

In [7]:
def build_squeezenet(shape=(224, 224, 3), classes=1000):
  # input tensor
  inputs = keras.Input(shape=shape)
  # stem component
  x = stem(inputs)
  # learner component
  x = learner(x)
  # task component
  outputs = task(x, classes)

  # return the SqueezeNet model
  return keras.Model(inputs=inputs, outputs=outputs, name="SqueezeNet")

In [8]:
# create the SqueezeNet model
squeezenet_model = build_squeezenet()
# print a summary
squeezenet_model.summary()

Model: "SqueezeNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 96)         14208     ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 55, 55, 96)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 55, 55, 16)           1552      ['max_pooling2d[0][0]